## Kaggle Model

For the sake of time, I based my kaggle submission on the work I had completed on my baseline model of lot features only. 

In [1]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, LassoCV
from sklearn import metrics

In [2]:
# read in the datasets
train = pd.read_csv('../datasets/train.csv')
test = pd.read_csv('../datasets/test.csv')

In [7]:
# change this commonly used grading scale from str to int
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.replace.html
num_scale = {
        'Ex':int(5),
        'Gd':int(4),
        'TA':int(3),
        'Fa':int(2),
        'Po':int(1)
}

train.replace(to_replace = num_scale, value = None, inplace = True)
test.replace(to_replace = num_scale, value = None, inplace = True)

In [8]:
#create a dataframe focused on features of the land itself
train_lots = pd.DataFrame(train, columns = ['Id', 'SalePrice', 'MS Zoning', 'Lot Area', 'Lot Shape', 'Land Contour', 'Lot Config', 'Neighborhood'])
test_lots = pd.DataFrame(test, columns = ['Id', 'MS Zoning', 'Lot Area', 'Lot Shape', 'Land Contour', 'Lot Config', 'Neighborhood'])

In [9]:
train_lots = pd.get_dummies(data = train_lots, columns = ['MS Zoning', 'Lot Shape', 'Land Contour', 'Lot Config', 'Neighborhood'], drop_first = True)
test_lots = pd.get_dummies(data = test_lots, columns = ['MS Zoning', 'Lot Shape', 'Land Contour', 'Lot Config', 'Neighborhood'], drop_first = True)

In [10]:
missing_cols = set(train_lots.columns) - set(test_lots.columns)
for c in missing_cols:
    test_lots[c] = 0
test_lots = test_lots[train_lots.columns]

missing_cols2 = set(test_lots.columns) - set(train_lots.columns)
for c in missing_cols2:
    train_lots[c] = 0
train_lots = train_lots[test_lots.columns]

test_lots.drop(columns = 'SalePrice', inplace = True)
#https://stackoverflow.com/questions/41335718/keep-same-dummy-variable-in-training-and-testing-data

In [13]:
X_lots = train_lots.drop(columns = ['SalePrice'])
y_lots = train_lots['SalePrice']

In [15]:
model_lots = LinearRegression()
model_lots.fit(X_lots, y_lots)

LinearRegression()

In [19]:
kaggle_preds = model_lots.predict(test_lots)

In [22]:
kaggle_preds.shape

(878,)

In [23]:
test_lots.shape

(878, 45)

In [24]:
kaggle_output = {'Id':test['Id'], 'SalePrice':kaggle_preds}

In [25]:
kaggle_output = pd.DataFrame(kaggle_output)

In [26]:
kaggle_output.to_csv('./demo_submission.csv', index = False) # Be sure to set index = False or kaggle will reject